In [1]:
from scipy.stats import beta
from scipy.optimize import fmin
from scipy.special import betaln
import numpy as np

## Setup

In [2]:
# Set up
RANDOM_SEED = 888
SIM_SIZE = 100000
CI_PROB = 0.95

success_a = 100
total_a = 250
success_b = 130
total_b = 300

In [3]:
# Success rates
rate_a = success_a / total_a
rate_b = success_b / total_b

print(rate_a, rate_b, rate_b-rate_a)

0.4 0.43333333333333335 0.033333333333333326


In [4]:
# Alpha/Beta parameters
alpha_a = success_a + 1
beta_a = total_a - success_a + 1
alpha_b = success_b + 1
beta_b = total_b - success_b + 1

print(alpha_a, beta_a, alpha_b, beta_b)

101 151 131 171


## Highest (Posterior) Density Intervals

Narrowest interval that captures the 95% of values that are most likely to occur

In [5]:
# Find Highest Density Interval from an Inverse Cumulative Distribution Function
# Adapted from: Doing Bayesian Data Analysis by John K. Kruschke
def hdi_from_icdf(dist_name, ci_prob, **args):
    
    dist = dist_name(**args)
    
    def interval_width(low_prob):
        return dist.ppf(low_prob+ci_prob) - dist.ppf(low_prob)
    
    initial_guess = 1.0 - ci_prob
    ci_low_prob = fmin(interval_width, initial_guess, ftol=1e-8, disp=False)[0]
    
    return dist.ppf([ci_low_prob, ci_low_prob+ci_prob])

In [6]:
# HDI for A and B
hdi_a = hdi_from_icdf(beta, ci_prob=CI_PROB, a=alpha_a, b=beta_a)
hdi_b = hdi_from_icdf(beta, ci_prob=CI_PROB, a=alpha_b, b=beta_b)

print(hdi_a, hdi_b)

[0.34068584 0.46133168] [0.37814769 0.48966612]


In [7]:
# Find Highest Density Interval from a sample
# Check all 95% widths and pick the one that is smallest
def hdi_from_values(sample, ci_prob):
    
    sorted_sample = np.sort(sample)
    
    sample_size = len(sorted_sample)
    ci_size = min(int(np.ceil(sample_size*0.95)), sample_size-1)
    
    num_ci = sample_size - ci_size
    ci_widths = np.repeat(0.0, num_ci)
    for i in range(num_ci):
        ci_widths[i] = sorted_sample[i+ci_size] - sorted_sample[i]
        
    hdi_low = sorted_sample[ci_widths.argmin()]
    hdi_upp = sorted_sample[ci_widths.argmin() + ci_size]
    
    return [hdi_low, hdi_upp]

In [8]:
# Simulate a sample of rate differences
diff_sample = beta.rvs(alpha_b, beta_b, size=SIM_SIZE, random_state=RANDOM_SEED) \
                - beta.rvs(alpha_a, beta_a, size=SIM_SIZE, random_state=RANDOM_SEED)

In [9]:
# HDI for Diff
hdi_from_values(diff_sample, ci_prob=CI_PROB)

[-0.040244199794189195, 0.10915908544664593]

## Probability that B is better than A

In [10]:
# Probability that B is better than A
# http://www.evanmiller.org/bayesian-ab-testing.html#binary_ab
def prob_b_beats_a(alpha_a, beta_a, alpha_b, beta_b):
    prob = 1.0
    for i in range(alpha_a):
        prob -= np.exp(betaln(alpha_b+i, beta_b+beta_a) \
                        - np.log(beta_a+i) - betaln(1+i, beta_a) - betaln(alpha_b, beta_b))
    return prob

In [11]:
prob_b_winner = prob_b_beats_a(alpha_a, beta_a, alpha_b, beta_b)
prob_b_winner

0.7840958578240851

In [12]:
prob_a_winner = prob_b_beats_a(alpha_b, beta_b, alpha_a, beta_a)
prob_a_winner

0.21590414217575646

## Expected loss if we choose B but A is actually better

Call a winner when expected loss is below some defined threshold

In [13]:
# Expected loss if choose B but A is actually better
# https://www.chrisstucchio.com/blog/2014/bayesian_ab_decision_rule.html

def expected_loss_b_over_a(alpha_a, beta_a, alpha_b, beta_b):
    exp_loss = np.exp(betaln(alpha_a+1, beta_a) - betaln(alpha_a, beta_a) \
                      + np.log(1-prob_b_beats_a(alpha_a+1, beta_a, alpha_b, beta_b))) \
             - np.exp(betaln(alpha_b+1, beta_b) - betaln(alpha_b, beta_b) \
                      + np.log(1-prob_b_beats_a(alpha_a, beta_a, alpha_b+1, beta_b)))
    return exp_loss

In [14]:
exp_loss = expected_loss_b_over_a(alpha_a, beta_a, alpha_b, beta_b)
exp_loss

0.005185573911307043

In [15]:
exp_uplift = expected_loss_b_over_a(alpha_b, beta_b, alpha_a, beta_a)
exp_uplift

0.03816675755479232

### Alternative numerical method

In [16]:
posteriorA = beta.cdf(np.linspace(0.0,0.99,100)+0.01, alpha_a, beta_a)-beta.cdf(np.linspace(0.0,0.99,100), alpha_a, beta_a)
posteriorB = beta.cdf(np.linspace(0.0,0.99,100)+0.01, alpha_b, beta_b)-beta.cdf(np.linspace(0.0,0.99,100), alpha_b, beta_b)

In [17]:
joint_posterior = np.zeros([100,100])

In [18]:
for i in range(100):
    for j in range(100):
        joint_posterior[i,j] = posteriorA[i] * posteriorB[j]

In [19]:
# Probability we make a mistake by choosing A
errorFunctionA = 0.0
for i in range(100):
    for j in range(i, 100):
        errorFunctionA += joint_posterior[i,j]
errorFunctionA

0.8166749959983718

In [20]:
# Probability we make a mistake by choosing B
errorFunctionB = 0.0
for i in range(100):
    for j in range(0, i):
        errorFunctionB += joint_posterior[i,j]
errorFunctionB

0.183325004001625

In [21]:
def loss(i, j, var):
    if var=='A':
        return max(j*0.01 - i*0.01, 0.0)
    if var=='B':
        return max(i*0.01 - j*0.01, 0.0)

In [22]:
# Expected loss for choosing B and we're wrong
lossFunction = 0.0
for i in range(100):
    for j in range(100):
        lossFunction += joint_posterior[i,j] * loss(i,j,'B')
lossFunction

0.005185573911297853

In [23]:
# Expected loss for choosing A and we're wrong
lossFunction = 0.0
for i in range(100):
    for j in range(100):
        lossFunction += joint_posterior[i,j] * loss(i,j,'A')
lossFunction

0.038166757554733005